# 0. Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import classifiers
import importlib
import json
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

# 1. Data Load

In [ ]:
reactome_hypergeometric = np.array(pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_closeness = np.array(pd.read_csv("../../data/processed/metrics/process_closeness_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_betweenness = np.array(pd.read_csv("../../data/processed/metrics/process_betweenness_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_rwr = np.array(pd.read_csv("../../data/processed/metrics/process_rwr_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_fraction_betweenness = np.array(pd.read_csv("../../data/processed/metrics/process_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0).transpose())

#reactome_bridge = np.array(np.array_split(pd.read_csv("../../data/processed/fs/related/reactome_bridge_fs_apid_huri.csv", sep=',', header=0).transpose(),429,axis=0))
#reactome_scp = np.array(np.array_split(pd.read_csv("../../data/processed/fs/related/reactome_scp_fs_apid_huri.csv", sep=',', header=0).transpose(),429,axis=0))
#reactome_slb = np.array(np.array_split(pd.read_csv("../../data/processed/fs/related/reactome_slb_fs_apid_huri.csv", sep=',', header=0).transpose(),429,axis=0))

reactome_hypergeometric_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)

#reactome_bridge_80 = np.array_split(np.load('../../data/processed/fs/reactome_bridge_80_fs_apid_huri.npz', allow_pickle=True)['arr_0'],10,axis=0)
#reactome_scp_80 = np.array_split(np.load('../../data/processed/fs/reactome_scp_80_fs_apid_huri.npz', allow_pickle=True)['arr_0'],10,axis=0)

In [ ]:
reactome_labels_df = pd.read_csv('../../data/processed/reactome_labels_apid_huri.csv', header=None)
reactome_labels = reactome_labels_df.transpose().to_numpy(dtype='int')
reactome_labels_df.columns=pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0).columns
reactome_labels_df.index=pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0).index

In [ ]:
ppi80_hyper_list = []
ppi80_closeness_list = []
ppi80_betweenness_list = []
ppi80_fraction_betweenness_list = []
ppi80_rwr_list = []
label_list = []

for i, df in reactome_hypergeometric_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list.append(df)
    label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])

for i, df in reactome_closeness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list.append(df)
    
for i, df in reactome_betweenness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list.append(df)
    
for i, df in reactome_fraction_betweenness_80.groupby(level=0):
    df.set_index('index', inplace=True)
    ppi80_fraction_betweenness_list.append(df)
    
for i, df in reactome_rwr_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list.append(df)


In [ ]:
ppi80_bridge_list = []
ppi80_scp_list = []

for df in range(len(reactome_bridge_80)):
    bridge = reactome_bridge_80[df][~np.any(np.isnan(reactome_bridge_80[df]), axis=1)]
    ppi80_bridge_list.append(np.array_split(bridge, 429, axis=1))
    
for df in range(len(reactome_bridge_80)):
    scp = reactome_bridge_80[df][~np.any(np.isnan(reactome_bridge_80[df]), axis=1)]
    ppi80_scp_list.append(np.array_split(scp, 429, axis=1))

In [ ]:
#reactome_hypergeometric_fs_ar = pd.read_csv("../../data/processed/fs/reactome_hyper_fs_apid_huri.csv", sep=',', header=0)
#reactome_closeness_fs_ar = pd.read_csv("../../data/processed/fs/reactome_closeness_fs_apid_huri.csv", sep=',', header=0)
#reactome_betweenness_fs_ar = pd.read_csv("../../data/processed/fs/reactome_betweenness_fs_apid_huri.csv", sep=',', header=0)
#reactome_rwr_fs_ar = pd.read_csv("../../data/processed/fs/reactome_rwr_fs_apid_huri.csv", sep=',', header=0)
#reactome_fraction_betweenness_fs_ar = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_fs_apid_huri.csv", sep=',', header=0)

reactome_hypergeometric_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_closeness_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_betweenness_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_rwr_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_fraction_betweenness_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))

In [ ]:
#reactome_hypergeometric_test_index = pd.read_csv("../../data/processed/fs/reactome_hyper_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
#reactome_closeness_test_index = pd.read_csv("../../data/processed/fs/reactome_closeness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
#reactome_betweenness_test_index = pd.read_csv("../../data/processed/fs/reactome_betweenness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
#reactome_rwr_test_index = pd.read_csv("../../data/processed/fs/reactome_rwr_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
#reactome_fraction_betweenness_test_index = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

reactome_hypergeometric_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_closeness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_rwr_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_fraction_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

In [ ]:
reactome_hypergeometric_80_test_index.shape

# 2. Classification Tasks

In [ ]:
def process_selector(fs_df, method):
    fs =[]
    for column in range(len(fs_df.columns)):
        process = fs_df.columns[column]
        if method == 'outlier':
            Q3 = fs_df[process].quantile(0.75)
            Q1 = fs_df[process].quantile(0.25)
            IQR = Q3 - Q1
            threshold = Q3 + 1.5*IQR
            if len(fs_df[fs_df[process]>threshold].index) > 0:
                fs.append(np.array(fs_df[fs_df[process]>threshold].index))
            else:
                fs.append(np.array(fs_df[fs_df[process]>=1].index))
        if method == 'outlier10':
            Q3 = fs_df[process].quantile(0.75)
            Q1 = fs_df[process].quantile(0.25)
            IQR = Q3 - Q1
            threshold = Q3 + 1.5*IQR
            if len(fs_df[fs_df[process]>threshold].index) > 0:
                fs.append(np.array(fs_df[fs_df[process]>threshold].index))
            else:
                fs.append(np.array(list(fs_df[process].sort_values(ascending=False)[:11].index)))
        if method == 'outlier/Q3':
            Q3 = fs_df[process].quantile(0.75)
            Q1 = fs_df[process].quantile(0.25)
            IQR = Q3 - Q1
            threshold = Q3 + 1.5*IQR
            if len(fs_df[fs_df[process]>threshold].index) > 0:
                fs.append(np.array(fs_df[fs_df[process]>threshold].index))
            else:
                fs.append(np.array(fs_df[fs_df[process]>Q3].index))
        if method == '10':
            fs_list = list(fs_df[process].sort_values(ascending=False)[:11].index)
            if column not in fs_list:
                fs_list = fs_list[:10]
                fs_list.append(column)
            fs.append(fs_list)
    return np.array(fs)

## 2.1. Classification with 11 (10 + self) best processes

### 2.1.1. Complete Network

In [ ]:
importlib.reload(classifiers)

clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]

reactome_hypergeometric_fs = process_selector(reactome_hypergeometric_fs_ar, '10')
hyper_lgr_clf, hyper_lgr_proba_clf, hyper_best_parameters = classifiers.classifier(clf, parameters, reactome_hypergeometric, reactome_hypergeometric_test_index, reactome_hypergeometric_fs, pd.DataFrame(reactome_labels).transpose())
hyper_lgr_clf.to_csv('../../models/apid_huri/process_hyper_lgr.csv', index=False)
hyper_lgr_proba_clf.to_csv('../../models/apid_huri/process_hyper_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/hyper_best_parameters', 'w') as fout:
    json.dump(hyper_best_parameters, fout)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
reactome_rwr_fs = process_selector(reactome_rwr_fs_ar, '10')
rwr_lgr_clf, rwr_lgr_proba_clf, rwr_best_parameters = classifiers.classifier(clf, parameters, reactome_rwr, reactome_rwr_test_index, reactome_rwr_fs, reactome_labels)
rwr_lgr_clf.to_csv('../../models/apid_huri/process_rwr_lgr.csv', index=False)
rwr_lgr_proba_clf.to_csv('../../models/apid_huri/process_rwr_lgr2.csv', index = False)
with open('../../models/apid_huri/best_parameters/rwr_best_parameters', 'w') as fout:
    json.dump(rwr_best_parameters, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
reactome_closeness_fs = process_selector(reactome_closeness_fs_ar, '10')
closeness_lgr_clf, closeness_lgr_proba_clf, closeness_best_parameters = classifiers.classifier(clf, parameters, reactome_closeness, reactome_closeness_test_index, reactome_closeness_fs, reactome_labels)
closeness_lgr_clf.to_csv('../../models/apid_huri/process_closeness_lgr.csv', index=False)
closeness_lgr_proba_clf.to_csv('../../models/apid_huri/process_closeness_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/closeness_best_parameters', 'w') as fout:
    json.dump(closeness_best_parameters, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
reactome_betweenness_fs = process_selector(reactome_betweenness_fs_ar, '10')
betweenness_lgr_clf, betweenness_lgr_proba_clf, betweenness_best_parameters = classifiers.classifier(clf, parameters, reactome_betweenness, reactome_betweenness_test_index, reactome_betweenness_fs, reactome_labels)
betweenness_lgr_clf.to_csv('../../models/apid_huri/process_betweenness_lgr.csv', index=False)
betweenness_lgr_proba_clf.to_csv('../../models/apid_huri/process_betweenness_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/betweenness_best_parameters', 'w') as fout:
    json.dump(betweenness_best_parameters, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
reactome_fraction_betweenness_fs = process_selector(reactome_fraction_betweenness_fs_ar, '10')
fraction_betweenness_lgr_clf, fraction_betweenness_lgr_proba_clf, fraction_betweenness_best_parameters = classifiers.classifier(clf, parameters, reactome_fraction_betweenness, reactome_fraction_betweenness_test_index, reactome_fraction_betweenness_fs, pd.DataFrame(reactome_labels).transpose())
fraction_betweenness_lgr_clf.to_csv('../../models/apid_huri/process_fraction_betweenness_lgr.csv', index=False)
fraction_betweenness_lgr_proba_clf.to_csv('../../models/apid_huri/process_fraction_betweenness_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/fraction_betweenness_best_parameters', 'w') as fout:
    json.dump(fraction_betweenness_best_parameters, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
bridge_lgr_clf, bridge_lgr_proba_clf, bridge_best_parameters = classifiers.classifier(clf, parameters, reactome_bridge, reactome_bridge_test_index, None, reactome_labels, related=True)
bridge_lgr_clf.to_csv('../../models/apid_huri/process_bridge_lgr.csv', index=False)
bridge_lgr_proba_clf.to_csv('../../models/apid_huri/process_bridge_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/bridge_best_parameters', 'w') as fout:
    json.dump(bridge_best_parameters, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
scp_lgr_clf, scp_lgr_proba_clf, scp_best_parameters = classifiers.classifier(clf, parameters, reactome_scp, reactome_scp_test_index, None, reactome_labels, related=True)
scp_lgr_clf.to_csv('../../models/apid_huri/process_scp_lgr.csv', index=False)
scp_lgr_proba_clf.to_csv('../../models/apid_huri/process_scp_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/scp_best_parameters', 'w') as fout:
    json.dump(scp_best_parameters, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
slb_lgr_clf, slb_lgr_proba_clf, slb_best_parameters = classifiers.classifier(clf, parameters, reactome_slb, reactome_slb_test_index, None, reactome_labels, related=True)
slb_lgr_clf.to_csv('../../models/apid_huri/process_slb_lgr.csv', index=False)
slb_lgr_proba_clf.to_csv('../../models/apid_huri/process_slb_lgr2.csv', index=False)
with open('../../models/apid_huri/best_parameters/slb_best_parameters', 'w') as fout:
    json.dump(slb_best_parameters, fout)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
hyper_lgr_clf3 = classifiers.classifier3(clf, hyper_best_parameters, reactome_hypergeometric, reactome_hypergeometric_test_index, reactome_hypergeometric_fs_ar, reactome_labels, hyper_lgr_proba_clf['threshold'].median())
hyper_lgr_clf3.to_csv('../../models/apid_huri/process_hyper_lgr3.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
rwr_lgr_clf3 = classifiers.classifier3(clf, rwr_best_parameters, reactome_rwr, reactome_rwr_test_index, reactome_rwr_fs_ar, reactome_labels, rwr_lgr_proba_clf['threshold'].median())
rwr_lgr_clf3.to_csv('../../models/apid_huri/process_rwr_lgr3.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
closeness_lgr_clf3 = classifiers.classifier3(clf, closeness_best_parameters, reactome_closeness, reactome_closeness_test_index, reactome_closeness_fs_ar, reactome_labels, closeness_lgr_proba_clf['threshold'].median())
closeness_lgr_clf3.to_csv('../../models/apid_huri/process_closeness_lgr3.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
betweenness_lgr_clf3 = classifiers.classifier3(clf, betweenness_best_parameters, reactome_betweenness, reactome_betweenness_test_index, reactome_betweenness_fs_ar, reactome_labels, betweenness_lgr_proba_clf['threshold'].median())
betweenness_lgr_clf3.to_csv('../../models/apid_huri/process_betweenness_lgr3.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
bridge_lgr_clf3 = classifiers.classifier3(clf, bridge_best_parameters, reactome_bridge, reactome_bridge_test_index, None, reactome_labels, bridge_lgr_proba_clf['threshold'].median(), related=True)
bridge_lgr_clf3.to_csv('../../models/apid_huri/process_bridge_lgr3.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
scp_lgr_clf3 = classifiers.classifier3(clf, scp_best_parameters, reactome_scp, reactome_scp_test_index, None, reactome_labels, scp_lgr_proba_clf['threshold'].median(), related=True)
scp_lgr_clf3.to_csv('../../models/apid_huri/process_scp_lgr3.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
slb_lgr_clf3 = classifiers.classifier3(clf, slb_best_parameters, reactome_slb, reactome_slb_test_index, None, reactome_labels, slb_lgr_proba_clf['threshold'].median(), related=True)
slb_lgr_clf3.to_csv('../../models/apid_huri/process_slb_lgr3.csv', index=False)

### 2.1.2. Reduced Networks

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
hyper_lgr_clf4, hyper_lgr_proba_clf4, hyper_lgr_clf4_bp = classifiers.reduced_classifiers(1, clf, parameters, ppi80_hyper_list, reactome_hypergeometric_80_test_index, reactome_hypergeometric_80_fs, reactome_labels_df)
hyper_lgr_clf4.to_csv('../../models/apid_huri/process_hyper_80_lgr.csv', index=True)
hyper_lgr_proba_clf4.to_csv('../../models/apid_huri/process_hyper_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/hyper_80_best_parameters', 'w') as fout:
    json.dump(hyper_lgr_clf4_bp, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_lgr_clf4, rwr_lgr_proba_clf4, rwr_lgr_clf4_bp = classifiers.reduced_classifiers(1, clf, parameters, ppi80_rwr_list, reactome_rwr_80_test_index, reactome_rwr_80_fs, reactome_labels_df)
rwr_lgr_clf4.to_csv('../../models/apid_huri/process_rwr_80_lgr.csv', index=True)
rwr_lgr_proba_clf4.to_csv('../../models/apid_huri/process_rwr_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/rwr_80_best_parameters', 'w') as fout:
    json.dump(hyper_lgr_clf4_bp, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
closeness_lgr_clf4, closeness_lgr_proba_clf4, closeness_lgr_clf4_bp = reduced_classifiers(1, clf, parameters, ppi80_closeness_list, reactome_closeness_80_test_index, reactome_closeness_80_fs, reactome_labels_df)
closeness_lgr_clf4.to_csv('../../models/apid_huri/process_closeness_80_lgr.csv', index=True)
closeness_lgr_proba_clf4.to_csv('../../models/apid_huri/process_closeness_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/closeness_80_best_parameters', 'w') as fout:
    json.dump(closeness_lgr_clf4_bp, fout)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
betweenness_lgr_clf4, betweenness_lgr_proba_clf4, betweenness_lgr_clf4_bp = classifiers.reduced_classifiers(1, clf, parameters, ppi80_betweenness_list, reactome_betweenness_80_test_index, reactome_betweenness_80_fs_ar, label_list)
betweenness_lgr_clf4.to_csv('../../models/apid_huri/process_betweenness_80_lgr.csv', index=True)
betweenness_lgr_proba_clf4.to_csv('../../models/apid_huri/process_betweenness_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/betweenness_80_best_parameters', 'w') as fout:
    json.dump(closeness_lgr_clf4_bp, fout)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
fraction_betweenness_lgr_clf4, fraction_betweenness_lgr_proba_clf4, fraction_betweenness_lgr_clf4_bp = classifiers.reduced_classifiers(1, clf, parameters, ppi80_fraction_betweenness_list, reactome_fraction_betweenness_80_test_index, reactome_fraction_betweenness_80_fs_ar, label_list)
fraction_betweenness_lgr_clf4.to_csv('../../models/apid_huri/process_fraction_betweenness_80_lgr.csv', index=True)
fraction_betweenness_lgr_proba_clf4.to_csv('../../models/apid_huri/process_fraction_betweenness_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/fraction_betweenness_80_best_parameters', 'w') as fout:
    json.dump(fraction_betweenness_lgr_clf4_bp, fout)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
bridge_lgr_clf4, bridge_lgr_proba_clf4, bridge_lgr_clf4_bp = classifiers.reduced_classifiers(1, clf, parameters, ppi80_bridge_list, reactome_bridge_80_test_index, None, label_list, related=True)
bridge_lgr_clf4.to_csv('../../models/apid_huri/process_bridge_80_lgr.csv', index=True)
bridge_lgr_proba_clf4.to_csv('../../models/apid_huri/process_bridge_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/bridge_80_best_parameters', 'w') as fout:
    json.dump(bridge_lgr_clf4_bp, fout)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
scp_lgr_clf4, scp_lgr_proba_clf4, scp_lgr_clf4_bp = reduced_classifiers(1, clf, parameters, ppi80_scp_list, reactome_scp_80_test_index, None, label_list)
scp_lgr_clf4.to_csv('../../models/apid_huri/process_scp_80_lgr.csv', index=True)
scp_lgr_proba_clf4.to_csv('../../models/apid_huri/process_scp_80_lgr2.csv', index=True)
with open('../../models/apid_huri/best_parameters/scp_80_best_parameters', 'w') as fout:
    json.dump(scp_lgr_clf4_bp, fout)